# Modelo bayesiano

Variables

Fuente_Industrial (Discreta: Baja/Alta)
Velocidad_Corriente (Discreta: Lenta/Rápida)
Profundidad (Discreta: Superficie/Fondo)
Concentracion_Mercurio (Discreta: Baja/Media/Alta)
Impacto_Ecologico (Binaria: Sí/No)
Riesgo_Humano (Binaria: Seguro/Peligroso)

Establecer Dependencias Causales


* Mayores emisiones industriales aumentan la concentración de mercurio.
* Corrientes rápidas dispersan el contaminante más rápido.
* El mercurio puede sedimentarse en el fondo (profundidad).
* Impacto_Ecologico	Concentraciones altas dañan la biodiversidad.
* Niveles elevados de concentracion implican riesgo para la salud.

Utilizar datos_mercurio_seguros.csv para llenar la tabla de distribucion de probabilidades.

Consultas a responder con el modelo:

* Probabilidad de alto riesgo humano cuando la fuente industrial es alta
* Cuál es la probabilidad de que el riesgo humano sea peligroso cuando la concentración de mercurio es alta y la fuente industrial está activa?"
* ¿Cómo cambia el impacto ecológico si se reduce la velocidad de la corriente en zonas profundas?"
* "¿A partir de qué nivel de concentración la probabilidad de riesgo humano supera el 50%?"

In [1]:
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator

In [2]:
# =================================================================
# 1. DEFINICIÓN DEL MODELO BAYESIANO
# =================================================================
model = DiscreteBayesianNetwork([
    ('Fuente_Industrial', 'Concentracion_Mercurio'),
    ('Velocidad_Corriente', 'Concentracion_Mercurio'),
    ('Profundidad', 'Concentracion_Mercurio'),
    ('Concentracion_Mercurio', 'Impacto_Ecologico'),
    ('Concentracion_Mercurio', 'Riesgo_Humano')
])

In [3]:
# =================================================================
# 2. APRENDIZAJE DE LAS PROBABILIDADES DESDE DATOS
# =================================================================
# Cargar datos
data = pd.read_csv('datos_mercurio_seguros.csv')

# Mapear valores numéricos a etiquetas textuales
mapping = {
    'Fuente_Industrial': {0: 'Baja', 1: 'Alta'},
    'Velocidad_Corriente': {0: 'Lenta', 1: 'Rápida'},
    'Profundidad': {0: 'Superficie', 1: 'Fondo'},
    'Concentracion_Mercurio': {0: 'Baja', 1: 'Media', 2: 'Alta'},
    'Impacto_Ecologico': {0: 'No', 1: 'Sí'},
    'Riesgo_Humano': {0: 'Seguro', 1: 'Peligroso'}
}

# Aplicar el mapeo a todas las columnas
for col, mapping_dict in mapping.items():
    if col in data.columns:
        data[col] = data[col].map(mapping_dict)

# Aprender las CPDs de los datos
model.fit(data, estimator=MaximumLikelihoodEstimator)

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'Fuente_Industrial': 'C', 'Velocidad_Corriente': 'C', 'Profundidad': 'C', 'Concentracion_Mercurio': 'C', 'Impacto_Ecologico': 'C', 'Riesgo_Humano': 'C'}


In [4]:
# =================================================================
# 3. REALIZAR CONSULTAS
# =================================================================
inference = VariableElimination(model)

# Consulta 1: Probabilidad de alto riesgo humano cuando la fuente industrial es alta
query1 = inference.query(
    variables=['Riesgo_Humano'],
    evidence={'Fuente_Industrial': 'Alta'}
)
print("\n1. Probabilidad de riesgo humano peligroso | Fuente Industrial = Alta:")
print(query1)

# Consulta 2: Riesgo peligroso cuando concentración es alta y fuente industrial activa
query2 = inference.query(
    variables=['Riesgo_Humano'],
    evidence={
        'Concentracion_Mercurio': 'Alta',
        'Fuente_Industrial': 'Alta'
    }
)
print("\n2. Probabilidad riesgo peligroso | [Concentración=Alta y Fuente=Alta]:")
print(query2)

# Consulta 3: Cambio en impacto ecológico al reducir velocidad en zonas profundas
# Estado base: Velocidad rápida en fondo
query3_base = inference.query(
    variables=['Impacto_Ecologico'],
    evidence={
        'Velocidad_Corriente': 'Rápida',
        'Profundidad': 'Fondo'
    }
)

# Estado con reducción: Velocidad lenta en fondo
query3_reduced = inference.query(
    variables=['Impacto_Ecologico'],
    evidence={
        'Velocidad_Corriente': 'Lenta',
        'Profundidad': 'Fondo'
    }
)

print("\n3. Cambio en impacto ecológico al reducir velocidad en zonas profundas:")
print(f"• Con velocidad rápida: P(Impacto=Sí) = {query3_base.values[1]:.4f}")
print(f"• Con velocidad lenta: P(Impacto=Sí) = {query3_reduced.values[1]:.4f}")
print(f"• Diferencia: {query3_reduced.values[1] - query3_base.values[1]:.4f}")

# Consulta 4: Nivel de concentración donde riesgo humano supera 50%
print("\n4. Probabilidad de riesgo peligroso por nivel de concentración:")
for level in ['Baja', 'Media', 'Alta']:
    query4 = inference.query(
        variables=['Riesgo_Humano'],
        evidence={'Concentracion_Mercurio': level}
    )
    prob_peligroso = query4.values[1]
    print(f"• Concentración {level}: {prob_peligroso:.4f} {'(>50%)' if prob_peligroso > 0.5 else ''}")

# Encontrar el punto exacto donde supera 50%
threshold_level = None
for level in ['Baja', 'Media', 'Alta']:
    query4 = inference.query(
        variables=['Riesgo_Humano'],
        evidence={'Concentracion_Mercurio': level}
    )
    if query4.values[1] > 0.5:
        threshold_level = level
        break

print(f"\nEl riesgo supera 50% cuando la concentración es: {threshold_level}")


1. Probabilidad de riesgo humano peligroso | Fuente Industrial = Alta:
+--------------------------+----------------------+
| Riesgo_Humano            |   phi(Riesgo_Humano) |
+==========================+======================+
| Riesgo_Humano(Peligroso) |               0.2831 |
+--------------------------+----------------------+
| Riesgo_Humano(Seguro)    |               0.7169 |
+--------------------------+----------------------+

2. Probabilidad riesgo peligroso | [Concentración=Alta y Fuente=Alta]:
+--------------------------+----------------------+
| Riesgo_Humano            |   phi(Riesgo_Humano) |
+==========================+======================+
| Riesgo_Humano(Peligroso) |               0.5882 |
+--------------------------+----------------------+
| Riesgo_Humano(Seguro)    |               0.4118 |
+--------------------------+----------------------+

3. Cambio en impacto ecológico al reducir velocidad en zonas profundas:
• Con velocidad rápida: P(Impacto=Sí) = 0.4373
• Con ve

In [5]:
# =================================================================
# 4. VERIFICACIÓN DE LAS CPDs APRENDIDAS
# =================================================================
print("\n📊 TABLAS DE PROBABILIDAD CONDICIONAL APRENDIDAS (CPDs):")
for cpd in model.get_cpds():
    print(f"\n{cpd}")


📊 TABLAS DE PROBABILIDAD CONDICIONAL APRENDIDAS (CPDs):

+-------------------------+---------+
| Fuente_Industrial(Alta) | 0.29875 |
+-------------------------+---------+
| Fuente_Industrial(Baja) | 0.70125 |
+-------------------------+---------+

+-------------------------------+-----+-----------------------------+
| Fuente_Industrial             | ... | Fuente_Industrial(Baja)     |
+-------------------------------+-----+-----------------------------+
| Profundidad                   | ... | Profundidad(Superficie)     |
+-------------------------------+-----+-----------------------------+
| Velocidad_Corriente           | ... | Velocidad_Corriente(Rápida) |
+-------------------------------+-----+-----------------------------+
| Concentracion_Mercurio(Alta)  | ... | 0.1721311475409836          |
+-------------------------------+-----+-----------------------------+
| Concentracion_Mercurio(Baja)  | ... | 0.5655737704918032          |
+-------------------------------+-----+------------